In [ ]:
import torch
import os
from PIL import Image

from torch import nn  #导入神经网络模块
from torch.utils.data import DataLoader  #数据包管理工具，打包数据
from torch.utils.data import Dataset
from torchvision import transforms

import time

#from torchvision import  datasets  #封装了很多与图像相关的模型，数据集

'''检查 CUDA 是否可用，并设置设备（"cuda:0" 或 "cpu"）'''
print("PyTorch 版本：", torch.__version__)  # 打印 PyTorch 的版本号
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("设备：", device)  # 打印当前使用的设备
print("CUDA 可用：", torch.cuda.is_available())  # 打印 CUDA 是否可用
print("cuDNN 已启用：", torch.backends.cudnn.enabled)  # 打印 cuDNN 是否已启用
print("支持的 CUDA 版本：", torch.version.cuda)
print("cuDNN 版本：", torch.backends.cudnn.version())


class MyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): 数据集的根目录
            transform (callable, optional): 可选的图像变换
            标签为图片文件名的首字母，图片为bmp格式
        """
        self.root_dir = root_dir
        self.transform = transform

        # 收集所有图像路径和对应的标签（文件名的首字母）
        self.samples = []
        self.labels_set = set()  # 用于收集所有独特的标签

        # 遍历根目录下的所有bmp文件
        for img_name in os.listdir(root_dir):
            img_path = os.path.join(root_dir, img_name)
            if os.path.isfile(img_path) and img_name.lower().endswith('.bmp'):
                # 获取文件名的首字母作为标签
                first_letter = img_name[0].upper()  # 转换为大写以确保一致性
                self.samples.append((img_path, first_letter))
                self.labels_set.add(first_letter)

        # 创建标签到索引的映射
        self.classes = sorted(list(self.labels_set))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        # 更新samples中的标签为索引形式
        self.samples = [(img_path, self.class_to_idx[label]) for img_path, label in self.samples]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_idx = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label_idx

    def get_original_label(self, idx):
        """获取原始的首字母标签（字符串形式）"""
        img_path, label_idx = self.samples[idx]
        # 通过索引反向查找标签字符串
        for label, index in self.class_to_idx.items():
            if index == label_idx:
                return label
        return None


transform = transforms.Compose([
    transforms.Resize((28, 28)),  # 统一尺寸
    transforms.Grayscale(),  # 转为灰度（如果是彩色BMP）
    transforms.ToTensor(),  # 必须的
    #transforms.Normalize((0.1307,), (0.3081,))  # 可选，可以先注释掉
])
batch_size = 200
# 创建数据集
dataset = MyDataset(root_dir='data/1-Digit-TrainSet/TrainingSet', transform=transform)
#dataset = MyDataset(root_dir='data/1-Digit-TestSet/TestSet', transform=transform)
# 打印数据集信息
print(f"数据集大小: {len(dataset)}")
print(f"所有标签: {dataset.classes}")
print(f"标签映射: {dataset.class_to_idx}")

# 创建数据加载器
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
print("完成")
'''dataloader检查 必须num_workers=0'''
for images, labels in dataloader:
    print(f"图像形状: {images.shape}")
    print(f"标签: {labels}")
    break

# 获取原始标签示例
if len(dataset) > 0:
    original_label = dataset.get_original_label(0)
    print(f"第一个样本的原始标签: {original_label}")


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 使用padding=2来保持尺寸
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1)
        # 这样特征图尺寸就是 20 * 7 * 7
        self.fc1 = nn.Linear(20 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # 28x28 -> 28x28 (padding=2保持尺寸)
        x = torch.max_pool2d(x, 2)  # 28x28 -> 14x14
        x = torch.relu(self.conv2(x))  # 14x14 -> 14x14
        x = torch.max_pool2d(x, 2)  # 14x14 -> 7x7

        x = x.view(-1, 20 * 7 * 7)  # 展平
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 创建模型实例
if torch.cuda.is_available() == 1:
    device = torch.device("cuda")
    print("CNN is running on GPU")
else:
    device = torch.device("cpu")
    print("CNN is running on CPU")
model = SimpleCNN().to(device)  #把刚刚创建的模型传入到GPU

print(model)


def train(dataloader, model, loss_fn, optimizer):
    model.train()  #告诉模型，我要开始训练，模型中w进行随机化操作，已经更新w，在训练过程中，w会被修改的
    # pytorch提供2种方式来切换训练和测试的模式，分别是：model.train() 和 mdoel.eval()
    # 一般用法是：在训练开始之前写上model.train(),在测试时写上model.eval()
    batch_size_num = 1
    for X, y in dataloader:  #其中batch为每一个数据的编号
        X, y = X.to(device), y.to(device)  #把训练数据集和标签传入cpu或GPU
        pred = model.forward(X)  # .forward可以被省略，父类种已经对此功能进行了设置
        loss = loss_fn(pred, y)  # 通过交叉熵损失函数计算损失值loss
        # Backpropagation 进来一个batch的数据，计算一次梯度，更新一次网络
        optimizer.zero_grad()  # 梯度值清零
        loss.backward()  # 反向传播计算得到每个参数的梯度值w
        optimizer.step()  # 根据梯度更新网络w参数

        loss_value = loss.item()  # 从tensor数据种提取数据出来，tensor获取损失值
        if batch_size_num % 100 == 0:
            print(f"loss: {loss_value:>7f} [number:{batch_size_num}]")
        batch_size_num += 1


def Test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  #10000
    num_batches = len(dataloader)  # 打包的数量
    model.eval()  #测试，w就不能再更新
    test_loss, correct = 0, 0
    with torch.no_grad():  #一个上下文管理器，关闭梯度计算。当你确认不会调用Tensor.backward()的时候
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model.forward(X)
            test_loss += loss_fn(pred, y).item()  #test_loss是会自动累加每一个批次的损失值
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            a = (pred.argmax(1) == y)  #dim=1表示每一行中的最大值对应的索引号，dim=0表示每一列中的最大值对应的索引号
            b = (pred.argmax(1) == y).type(torch.float)
    test_loss /= num_batches  #能来衡量模型测试的好坏
    correct /= size  #平均的正确率
    print(f"Test result: \n Accuracy:{(100 * correct)}%, Avg loss:{test_loss}")


loss_fn = nn.CrossEntropyLoss()  #创建交叉熵损失函数对象，因为手写字识别一共有十种数字，输出会有10个结果
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  #创建一个优化器
#train(dataloader,model,loss_fn,optimizer)
#Test(test_dataloader,model,loss_fn)

In [ ]:

test_dataset = MyDataset(root_dir='data/1-Digit-TestSet/TestSet', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

epochs = 5
for t in range(epochs):
    print(f"epoch {t + 1}\n---------------")
    train(dataloader, model, loss_fn, optimizer)  #训练1次完整的数据。多轮训练
    Test(test_dataloader, model, loss_fn)
print("Done!")

test_dataset = MyDataset(root_dir='data/1-Digit-TestSet/TestSet', transform=transform)
# 打印数据集信息
print(f"数据集大小: {len(test_dataset)}")
print(f"所有标签: {test_dataset.classes}")
print(f"标签映射: {test_dataset.class_to_idx}")

# 创建数据加载器
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
print("完成")
Test(test_dataloader, model, loss_fn)